# 1. Types of descriptors
- 구현한 magic method가 무엇이냐에 따라 다음과 같이 구분
  - data descriptor
    - `__set__` 또는 `__delete__` 구현
    - descriptor와 동일한 이름의 attribute이 있는 경우 descriptor가 호출
  - non-data descriptor
    - `__get__`만 구현
    - descriptor와 동일한 이름의 attribute이 있는 경우 descriptor가 아닌 해당 객체를 호출
- `__set_name__`의 구현은 상관 없음

## 1.1. Non-data descriptors

In [12]:
class NonDataDescriptor:
    def __get__(self, instance, owner):
        print('descriptor __get__ called')
        if instance is None:
            return self

        return 42

In [13]:
class ClientClass:
    descriptor = NonDataDescriptor()

- `client` 객체를 생성시 descriptor는 instance attribute이 아닌 class attribute임
- 따라서 `vars(client)`를 실행 하면 빈 dictionary가 나오게 됨
- 또한 `.descriptor`를 호출하는 경우 `client.__dict__`에서 key를 찾을 수 없기 때문에 `class`에서 이를 찾으려고 하고 따라서 descriptor의 `__get__`이 호출되는것임

In [14]:
client = ClientClass()
client.descriptor

descriptor __get__ called


42

In [15]:
vars(client)

{}

* `.descriptor`에 다른 값을 할당하면, `__dict__`에 값이 할당됨
* 따라서 descriptor는 호출되지 않고 `__dict__['descriptor']` 값을 가져옴

In [16]:
client.descriptor = 43
client.descriptor

43

In [17]:
vars(client)

{'descriptor': 43}

* `del`로 attribute을 지우면 첫번째와 동일한 상황이 됨
* 따라서 descriptor protocol이 다시 동작함

In [18]:
del client.descriptor
client.descriptor

descriptor __get__ called


42

In [19]:
vars(client)

{}

## 1.2. Data descriptors

In [20]:
import logging

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)


class DataDescriptor:
    def __get__(self, instance, owner):
        print('descriptor __get__ called')
        if instance is None:
            return self
        return 42

    def __set__(self, instance, value):
        logger.debug("setting %s.descriptor to %s", instance, value)
        instance.__dict__["descriptor"] = value


class ClientClass:
    descriptor = DataDescriptor()


In [21]:
client = ClientClass()
client.descriptor

descriptor __get__ called


42

In [22]:
vars(client)

{}

- non-data descriptor와 다르게 변경된 값을 반환하지 않음
- attribute을 변경하더라도 descriptor `__get__`이 계속 호출됨
- 그러나 `__dict__`에 변경된 값이 들어가 있는건 맞음 (`__set__`에서 아래 line이 실행되니까)
  ```python
  instance.__dict__["descriptor"] = value
  ```

In [23]:
client.descriptor = 99
client.descriptor

setting <__main__.ClientClass object at 0x7f1059103690>.descriptor to 99


descriptor __get__ called


42

In [24]:
vars(client)

{'descriptor': 99}

- non-data descriptor와는 다르게 `del`로 attribute을 지울수 없음
- `del instanece.__dict__[attribute]`이 아닌 `instance.__delete__`가 호출됨
- `__set__`이 구현된 경우 항상 descriptor를 참조하게 됨!

In [25]:
del client.descriptor

AttributeError: ignored

In [30]:
import logging

logging.basicConfig(level=logging.DEBUG, format="%(message)s")
logger = logging.getLogger(__name__)


class DataDescriptorOld:
    def __get__(self, instance, owner):
        print('descriptor __get__ called')
        if instance is None:
            return self
        return 42

    def __set__(self, instance, value):
        logger.debug("setting %s.descriptor to %s", instance, value)
        instance.__dict__['descriptor'] = value

    def __set_name__(self, owner, name):
      self._name = name


class DataDescriptorNew:
    def __get__(self, instance, owner):
        print('descriptor __get__ called')
        if instance is None:
            return self
        return 42

    def __set__(self, instance, value):
        logger.debug("setting %s.descriptor to %s", instance, value)
        instance.__dict__[self._name] = value

    def __set_name__(self, owner, name):
      self._name = name


class DataDescriptorWorst:
    def __get__(self, instance, owner):
        print('descriptor __get__ called')
        if instance is None:
            return self
        return 42

    def __set__(self, instance, value):
        logger.debug("setting %s.descriptor to %s", instance, value)
        setattr(instance, self._name, value)

    def __set_name__(self, owner, name):
      self._name = name


class ClientClass:
    descriptor_old = DataDescriptorOld()
    descriptor_new = DataDescriptorNew()
    descriptor_worst = DataDescriptorWorst()

- 기타 참고할만한 내용들  
  - 아래 코드에서 `ClientClass`의 attribute name이 descriptor인지 어떻게 알았나?
    ```python
    instance.__dict__["descriptor"] = value
    ```
    - `Descriptor class`를 구현할때 `Client class`에서 **해당 `descriptor`가 어떤 `class attribute`으로 사용될지 아는건 말이 안됨**
    - 이런 고민은 `__set_name__`을 사용하면 쉽게 해결 가능

  - `__dict__`에 직접 접근하는것보다 `setattr(instance, 'descriptor', value)`가 더 나은 방법같은데?
    - `__set__`은 attribute에 특정 값을 할당할때 호출되는 것을 기억해야됨
    - 만약 `__set__`안에서 `setattr`을 사용하면 다시 `__set__`를 호출하게 돼서 **infinite recursion에 빠지게 됨**

In [31]:
client = ClientClass()
print(client.descriptor_old)
print(client.descriptor_new)

descriptor __get__ called
42
descriptor __get__ called
42


In [32]:
client.descriptor_old = 99
client.descriptor_new = 999
print(client.descriptor_old)
print(client.descriptor_new)

setting <__main__.ClientClass object at 0x7f1055e23f90>.descriptor to 99
setting <__main__.ClientClass object at 0x7f1055e23f90>.descriptor to 999


descriptor __get__ called
42
descriptor __get__ called
42


In [33]:
vars(client)

{'descriptor': 99, 'descriptor_new': 999}

In [35]:
# client.descriptor_worst = 9999